In [1]:
from jarvis.core.atoms import crop_square
import matplotlib.pyplot as plt
import os, glob
from jarvis.analysis.stem.convolution_apprx import STEMConv
from jarvis.db.figshare import data, get_jid_data
import matplotlib.pyplot as plt
from jarvis.core.atoms import Atoms, ase_to_atoms, get_supercell_dims
from jarvis.core.lattice import get_2d_lattice
from sklearn.model_selection import train_test_split
from collections import defaultdict
from jarvis.db.jsonutils import loadjson, dumpjson
from jarvis.core.atoms import Atoms
from pymatgen.core import Structure
from pymatgen.core.lattice import Lattice
import numpy as np

import json


In [2]:
#Read data

from datasets import load_dataset

data_train = load_dataset("json", data_files="./alpaca_mbj_bandgap_train.json", split="train")
data_test = load_dataset("json", data_files="./alpaca_mbj_bandgap_test.json", split="train")

/home/jipengsun/.conda/envs/clipp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Transform data to pandas DataFrame
train_df = data_train.to_pandas()
test_df = data_test.to_pandas()
# Save to CSV
#train_df.to_csv("alpaca_mbj_bandgap_train.csv", index=False)
#test_df.to_csv("alpaca_mbj_bandgap_test.csv", index=False)

In [4]:
def str_to_structure(gen_str):
    lines = [x for x in gen_str.split("\n") if len(x) > 0]
    lengths = [float(x) for x in lines[0].split(" ")]
    angles = [float(x) for x in lines[1].split(" ")]
    species = [x for x in lines[2::2]]
    coords = [[float(y) for y in x.split(" ")] for x in lines[3::2]]
    
    structure = Structure(
        lattice=Lattice.from_parameters(
            *(lengths + angles)),
        species=species,
        coords=coords, 
        coords_are_cartesian=False,
    )
    
    return structure

def structure_to_atoms(structure):
    import jarvis
    atoms = jarvis.core.atoms.pmg_to_atoms(structure)
    return atoms

def atom_to_image(atoms, size=128):
    """
    Convert Atoms object to a square image of given size.
    """
    dims = get_supercell_dims(atoms=atoms, enforce_c_size=50)
    s = atoms.make_supercell_matrix(dims)
    c = crop_square(s, csize=50)
    p = STEMConv(output_size=[size, size]).simulate_surface(atoms=c)[0]
    return p

# From list to image shape img to image
def list_to_image(img_list, size=128):
    """
    Convert a list to a 2D image of given size.
    """
    return json.loads(np.array(img_list).reshape(size, size))

In [8]:
# for i, row in train_df.iterrows():
#     gen_str = row['response']
    
#     structure = str_to_structure(gen_str)
#     atoms = structure_to_atoms(structure)
    
#     image = atom_to_image(atoms, size=128)

#     #save images on disk
#     image_path = f"train/train_image_{i}.png"
#     plt.imsave(image_path, image, cmap='gray')
    
#     train_df.loc[i, 'id'] =  i  # Add an ID column
#     train_df.loc[i, 'image'] = json.dumps(image.tolist())  # Store image as list for DataFrame

# train_df.to_csv("alpaca_mbj_bandgap_train.csv", index=False)

for i, row in test_df.iterrows():
    gen_str = row['response']
    
    structure = str_to_structure(gen_str)
    atoms = structure_to_atoms(structure)
    
    image = atom_to_image(atoms, size=128)

    #save images on disk
    image_path = f"test/test_image_{i}.png"
    plt.imsave(image_path, image, cmap='gray')

    test_df.loc[i, 'id'] =  i  # Add an ID column 
    test_df.loc[i, 'image'] = json.dumps(image.tolist())  # Store image as list for DataFrame

test_df.to_csv("alpaca_mbj_bandgap_test.csv", index=False)
    


/tmp/ipykernel_997221/1703519336.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[0.6020488765646538, 0.9551997471544535, 1.4831043832141149, 2.4976131368044237, 4.240968136447274, 6.77098375475415, 9.73475286889776, 12.31973260085934, 13.602868458627444, 13.006429561088257, 10.84578952337141, 8.022703198272422, 5.49833349609509, 3.8394490128449164, 3.1150296773075445, 3.039882329841961, 3.1937691019762555, 3.015943601428364, 3.117577460413179, 2.868195837695944, 2.8129711901741796, 3.270118502998278, 4.419926376957534, 6.153861228064575, 8.017092551765467, 9.358241589307733, 9.592232433007746, 8.587459669233537, 6.850967331405178, 5.038641477201766, 3.724010626479504, 3.1415116712519446, 3.1890128798848187, 3.543449871563384, 3.621232695669901, 3.641327364638783, 3.363081894192792, 2.908287369866485, 2.697798168534779, 2.965463001809884, 3.7536486356973953, 4.859553941774943, 5.876334528460158, 6.